In [ ]:
import pandas as pd
import plotnine as p9
import datetime as dt
import glob


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# custom convert date to match the garmin reported data
def convert_date(x):
    dates = []
    for d in x:
        dates.append(datetime.strptime(d['date'], "%b %d, %Y %I:%M:%S %p"))
    return dates

# extracts daily stress
def explode_daily_stress(x):
    ad = dict(x)
    al = ad['aggregatorList'][0]['averageStressLevel']
    return al

# extracts the awake stress score
def explode_daily_awake_stress(x):
    ad = dict(x)
    al = ad['aggregatorList'][1]['averageStressLevel']
    return al


In [ ]:
# get the 2021 uds files
uds_files = []
uds_df = pd.DataFrame()
for file in glob.glob("../data/f4617e72-eb4d-4f9d-8aec-8d965e6f651c_1/DI_CONNECT/DI-Connect-User/UDSFile*.json"):
    if '2021' in file: 
        temp_df = pd.read_json(file)
        uds_df = uds_df.append(temp_df)

uds_df.head(2)

# quick manipulations
uds_df['calendarDate'] = convert_date(uds_df['calendarDate'])
is_22W = uds_df.calendarDate >= dt.datetime.strptime('2022-01-4', '%Y-%m-%d')
uds_df = uds_df.loc[is_22W, :]
uds_df['AVG_DAILY_STRESS_DERIVED'] = uds_df.allDayStress.apply(lambda x: explode_daily_stress(x))
uds_df['AVG_AWAKE_STRESS_DERIVED'] = uds_df.allDayStress.apply(lambda x: explode_daily_awake_stress(x))

uds_df.info()

In [ ]:
uds_df['totalSteps'].describe()
uds_df.AVG_DAILY_STRESS_DERIVED.describe()
uds_df.restingHeartRate.describe()

In [ ]:
gourdotron_theme = (p9.theme_classic()
                   + p9.theme(axis_text_x = p9.element_text(angle=-45, color='#14a0ec', ha='left'), 
                           axis_text_y = p9.element_text(color='#14a0ec'))
                    + p9.theme(panel_background = p9.element_rect(fill='#444444'), 
                           plot_background = p9.element_rect(fill = "#444444"), 
                           axis_title_x=p9.element_text(color='#14a0ec'), 
                           axis_title_y=p9.element_text(color='#14a0ec')))

stress_long = (p9.ggplot(data=uds_df, mapping=p9.aes(x='calendarDate', y='AVG_DAILY_STRESS_DERIVED'))
                + p9.geom_line(color='#14a0ec')
                + p9.geom_area(fill="#F85621", alpha=0.8)
                + p9.geom_hline(mapping=p9.aes(yintercept=uds_df['AVG_DAILY_STRESS_DERIVED'].mean()), 
                                linetype='dashed', color='#14a0ec')
                + p9.geom_point(color='#F85621', stat='identity', fill='#F85621')
                + p9.labs(x='Calendar Date', y = 'Average Daily Stress')
                + gourdotron_theme)

stress_long